In [1]:
import sys
sys.path.append("..")

In [2]:
import plap
from plap.parameterization.parameterizer import Parameterizer

preprocessor = plap.Preprocessor(window_type="hamming")
parameterizer = Parameterizer(audio_path="../tests/data/redhot.wav", preprocessor=preprocessor)

parameterizer.signal.shape, parameterizer.sample_rate

INIT MPEG7: Unhandled case


((661500,), 22050)

# Parameterize Speech DB

In [7]:
import os
import numpy as np
import plap

# Define the paths to the subdirectories
male_folder = './audios/Speech/Male'
female_folder = './audios/Speech/Female'

# Initialize a list to hold the file names
all_files = []

# Get the list of all files in the Male folder and collect filenames
for file_name in os.listdir(male_folder):
    file_path = os.path.join(male_folder, file_name)
    if os.path.isfile(file_path):  # Check if it's a file
        all_files.append(file_path)

# Get the list of all files in the Female folder and collect filenames
for file_name in os.listdir(female_folder):
    file_path = os.path.join(female_folder, file_name)
    if os.path.isfile(file_path):  # Check if it's a file
        all_files.append(file_path)

# Sort the list of files alphabetically
all_files = sorted(all_files)

# Display the collected file names
# print('List of all files in Male and Female folders:')
# for file in all_files:
#     print(file)

# Use plap for parameterization
fvector = plap.FeatureVector('LAT','TC')

all_fvector = np.zeros((len(all_files), len(fvector.features)))

i = 0
for file in all_files:
    print(f"Processing file: {i+1}")
    plap.parameterize(
        audio_path=file,
        fvector=fvector,
        preprocessor=preprocessor
    )
    all_fvector[i,:] = fvector.values
    i += 1

all_fvector

Processing file: 1
Processing file: 2
Processing file: 3
Processing file: 4
Processing file: 5
Processing file: 6
Processing file: 7
Processing file: 8
Processing file: 9
Processing file: 10
Processing file: 11
Processing file: 12
Processing file: 13
Processing file: 14
Processing file: 15
Processing file: 16
Processing file: 17
Processing file: 18
Processing file: 19
Processing file: 20
Processing file: 21
Processing file: 22
Processing file: 23
Processing file: 24
Processing file: 25
Processing file: 26
Processing file: 27
Processing file: 28
Processing file: 29
Processing file: 30
Processing file: 31
Processing file: 32
Processing file: 33
Processing file: 34
Processing file: 35
Processing file: 36
Processing file: 37
Processing file: 38
Processing file: 39
Processing file: 40
Processing file: 41
Processing file: 42
Processing file: 43
Processing file: 44
Processing file: 45
Processing file: 46
Processing file: 47
Processing file: 48
Processing file: 49
Processing file: 50
Processin

array([[-0.27002571,  1.7670552 ],
       [-0.53443127,  1.09366817],
       [-0.52451041,  1.52100301],
       [ 0.15988663,  1.47945936],
       [ 0.15112076,  1.20342404],
       [-1.13928776,  1.0546202 ],
       [-0.40893539,  1.60538934],
       [-0.97112387,  1.45115033],
       [-1.07186029,  1.98631864],
       [ 0.34609614,  1.80114946],
       [ 0.0811447 ,  1.98452885],
       [-0.54802178,  1.16468542],
       [-0.88190069,  2.37869229],
       [ 0.42641976,  2.25062411],
       [ 0.39205911,  2.1221896 ],
       [-0.71542828,  1.7750555 ],
       [-0.44415013,  2.03366892],
       [-0.76843484,  0.69368814],
       [-0.68050356,  1.12137362],
       [-0.2562943 ,  0.55909906],
       [-0.05103718,  1.54613466],
       [-1.02456819,  1.21188861],
       [-0.16174138,  1.2317532 ],
       [ 0.23389995,  1.75786927],
       [-0.83657771,  1.94953822],
       [ 0.19624529,  1.15510414],
       [-1.00272286,  0.71918075],
       [-0.05204578,  1.11958383],
       [-0.97958692,

In [8]:
output_file = './python_csv/lat_tc.csv'
np.savetxt(output_file, all_fvector, delimiter=',')

# Compare implementations

In [10]:
import pandas as pd

def csv_values_to_numpy(file_path) -> np.ndarray:
    # Read the file and split values by commas
    array = np.loadtxt(file_path, delimiter=',')
    # Convert the data into a numpy array of floats
    numpy_array = np.array(array, dtype=float)
    return numpy_array

def compare_two_arrays(array1: np.ndarray, array2: np.ndarray, features: list):
    diff = np.abs(array1-array2)

    # Calculate mean, min, max, and std for each feature and store them
    mean_diffs = []
    min_diffs = []
    max_diffs = []
    mse_diffs = []
    offset = 0
    for feature in features:
        num_values = feature[1]
        mean_diffs.append(np.mean(diff[:, offset:offset+num_values]))
        min_diffs.append(np.min(diff[:, offset:offset+num_values]))
        max_diffs.append(np.max(diff[:, offset:offset+num_values]))
        mse_diffs.append(np.mean(diff[:, offset:offset+num_values]**2))
        offset += num_values
    
    # Create a dictionary with the collected data
    data = {
        'Mean difference': mean_diffs,
        'Min difference': min_diffs,
        'Max difference': max_diffs,
        'Mean Squared Error': mse_diffs,
    }

    # Create a DataFrame with the statistics
    feature_names = [f[0] for f in features]
    statistics_df = pd.DataFrame(data, index=feature_names).T
    return statistics_df

    # DIRECT COMPARISON FOR VERIFICATION
    # combined = np.column_stack((array1, array2, diff))
    # df = pd.DataFrame(combined)
    # print(df)
    # print(df.to_string())


## Timbral Temporal (LAT + TC) 

In [13]:
matlab_fv_path = './matlab_csv/lat_tc_sc_aff.csv'
matlab_feature_vector = csv_values_to_numpy(matlab_fv_path)
lat_tc_matlab = matlab_feature_vector[:,0:2]

lat_tc_python = all_fvector[:,0:2]
lat_tc_matlab.shape, lat_tc_python.shape


((278, 2), (278, 2))

In [14]:
features_list = [('lat', 1), ('tc', 1)]
stats = compare_two_arrays(lat_tc_matlab, lat_tc_python, features_list)
stats

,lat,tc
Mean difference,0.217923,0.052231
Min difference,0.000000,0.000067
Max difference,1.696840,0.292296
Mean Squared Error,0.170719,0.005123
